# Little Lemon Analysis and Sales Report

## Introduction
This project demonstrates the ability to interact with a MySQL database from a Python-based front-end application using MySQL Connector/Python. The aim is to create a comprehensive analysis and sales report for Little Lemon.

## Tasks Overview

### Task 1: Establish a Connection Pool
- **Objective**: Create a connection pool to manage database connections efficiently.
- **Steps**:
    1. Import `MySQLConnectionPool` and `Error` from `mysql.connector`.
    2. Define database configurations as a dictionary object called `dbconfig`.
    3. Establish a connection pool named `pool_b` with two connections.
    4. Implement error handling using a try-except block.

In [9]:
# Establish a Connection Pool

import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(
        host="localhost",
        user="----------",
        password="----------",
        database="little_lemon_db"
    )

    if connection.is_connected():
        print("Successfully connected to the database.")
except Error as err:
    print(f"Error: {err}")
finally:
    if connection.is_connected():
        connection.close()
        print("Connection closed.")


Successfully connected to the database.
Connection closed.


In [10]:
# Insert data into the Bookings table

def insert_guest_booking(table_no, first_name, last_name, booking_slot, employee_id):
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="----------",
            password="----------",
            database="little_lemon_db"
        )
        cursor = connection.cursor()
        
        insert_query = """
        INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
        VALUES (%s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (table_no, first_name, last_name, booking_slot, employee_id))
        connection.commit()
        print(f"Inserted booking for {first_name} {last_name}.")

    except Error as err:
        print(f"Error: {err}")

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

# Insert three guests with the correct column names
insert_guest_booking(8, "Anees", "Java", "18:00", 6)
insert_guest_booking(5, "Bald", "Vin", "19:00", 6)
insert_guest_booking(12, "Jay", "Kon", "19:30", 6)


Inserted booking for Anees Java.
Inserted booking for Bald Vin.
Inserted booking for Jay Kon.


In [11]:
# Create a report

import mysql.connector
from mysql.connector import Error

def generate_report():
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="----------",
            password="----------",
            database="little_lemon_db"
        )
        cursor = connection.cursor()

        # Query 1: Get the name and EmployeeID of the manager
        cursor.execute("SELECT Name, EmployeeID FROM Employees WHERE Role = 'Manager';")
        manager = cursor.fetchone()
        if manager:
            print("Manager:", manager[0], "EmployeeID:", manager[1])
        else:
            print("No manager found.")

        # Query 2: Get the employee with the highest salary
        cursor.execute("SELECT Name, Role FROM Employees ORDER BY CAST(Annual_Salary AS DECIMAL) DESC LIMIT 1;")
        highest_salary_employee = cursor.fetchone()
        if highest_salary_employee:
            print("Employee with highest salary:", highest_salary_employee[0], "Role:", highest_salary_employee[1])
        else:
            print("No employee found.")

        # Query 3: Count the number of guests booked between 18:00 and 20:00
        cursor.execute("SELECT COUNT(*) FROM Bookings WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00';")
        guest_count = cursor.fetchone()[0]
        print("Number of guests booked between 18:00 and 20:00:", guest_count)

        # Query 4: Full name and BookingID of all guests
        cursor.execute("SELECT BookingID, CONCAT(GuestFirstName, ' ', GuestLastName) AS FullName FROM Bookings ORDER BY BookingSlot;")
        all_guests = cursor.fetchall()
        print("Guests in the booking list:")
        for booking in all_guests:
            print(f"BookingID: {booking[0]}, Full Name: {booking[1]}")

    except Error as err:
        print(f"Error: {err}")

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

# Generate report
generate_report()


Manager: Mario Gollini EmployeeID: 1
Employee with highest salary: Mario Gollini Role: Manager
Number of guests booked between 18:00 and 20:00: 10
Guests in the booking list:
BookingID: 3, Full Name: Vanessa McCarthy
BookingID: 4, Full Name: Marcos Romero
BookingID: 7, Full Name: Anees Java
BookingID: 10, Full Name: Anees Java
BookingID: 5, Full Name: Hiroki Yamane
BookingID: 1, Full Name: Anna Iversen
BookingID: 2, Full Name: Joakim Iversen
BookingID: 8, Full Name: Bald Vin
BookingID: 11, Full Name: Bald Vin
BookingID: 9, Full Name: Jay Kon
BookingID: 12, Full Name: Jay Kon
BookingID: 6, Full Name: Diana Pinto


In [12]:
#Create a stored procedure named BasicSalesReport

import mysql.connector
from mysql.connector import Error

def execute_basic_sales_report():
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="----------",
            password="----------",
            database="little_lemon_db"
        )
        cursor = connection.cursor()

        # Call the stored procedure
        cursor.callproc('BasicSalesReport')

        # Fetch results
        for result in cursor.stored_results():
            total_sales, average_sale, minimum_bill, maximum_bill = result.fetchone()
            print("Total Sales:", total_sales)
            print("Average Sale:", average_sale)
            print("Minimum Bill:", minimum_bill)
            print("Maximum Bill:", maximum_bill)

    except Error as err:
        print(f"Error: {err}")

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

# Execute the function
execute_basic_sales_report()


Total Sales: 243
Average Sale: 48.6000
Minimum Bill: 37
Maximum Bill: 86


In [14]:
#Display upcoming bookings for kitchen notification

import mysql.connector
from mysql.connector import Error, pooling

# Define the connection pool
pool_a = pooling.MySQLConnectionPool(
    pool_name="mypool",
    pool_size=5,
    host="localhost",
    user="----------",
    password="----------",
    database="little_lemon_db"
)

# Display upcoming bookings for kitchen notification
def display_upcoming_bookings():
    connection = None  # Initialize connection variable
    try:
        # Get a connection from the pool
        connection = pool_a.get_connection()
        if connection.is_connected():
            cursor = connection.cursor(buffered=True)
            print("Connected to MySQL database")

            # SQL query to retrieve upcoming bookings
            query = """
            SELECT B.BookingSlot, CONCAT(B.GuestFirstName, ' ', B.GuestLastName) AS GuestName, 
                   CONCAT(E.Name, ' [', E.Role, ']') AS EmployeeInfo
            FROM Bookings B
            LEFT JOIN Employees E ON B.EmployeeID = E.EmployeeID
            ORDER BY B.BookingSlot ASC
            LIMIT 3;
            """

            # Execute the query
            cursor.execute(query)

            # Fetch the results
            results = cursor.fetchall()

            # Display the output in the specified format
            for row in results:
                booking_slot, guest_name, employee_info = row
                print(f"[{booking_slot}]\n[{guest_name}]\n[Assigned to: {employee_info}]\n")

    except Error as err:
        print(f"Error: {err}")

    finally:
        # Return the connection back to the pool
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is returned to the pool.")

# Call the function to display upcoming bookings
display_upcoming_bookings()



Connected to MySQL database
[15:00:00]
[Vanessa McCarthy]
[Assigned to: Giorgos Dioudis [Head Chef]]

[17:30:00]
[Marcos Romero]
[Assigned to: Fatma Kaya [Assistant Chef]]

[18:00:00]
[Anees Java]
[Assigned to: John Millar [Receptionist]]

MySQL connection is returned to the pool.
